In [48]:
actions = (0, 2, 0)
q_vals = np.arange(len(actions) * 4).reshape(-1, 4)
values = np.arange(len(actions)).reshape(-1, 1) * 10
n_visits = np.arange(len(actions) * 4).reshape(-1, 4)

In [61]:
still_playing = np.array([True, False, True])

In [62]:
print(np.argmax(q_vals, axis=1))
np.where(
    still_playing,
    np.argmax(q_vals, axis=1),
    -1
)

[3 2 0]


array([ 3, -1,  0])

In [53]:
q_vals = np.where(
    np.eye(4)[np.array(actions)],
    q_vals + values,
    q_vals
)
q_vals

array([[ 0,  1,  2,  3],
       [ 4,  5, 16,  7],
       [28,  9, 10, 11]])

In [1]:
import copy
from kaggle_environments import make as kaggle_make
import numpy as np
from scipy import stats
import torch
import torch.nn.functional as F

from hungry_geese.env import goose_env as ge
from hungry_geese.env.lightweight_env import LightweightEnv, make
from hungry_geese.mcts.basic_mcts import run_mcts

Loading environment football failed: No module named 'gfootball'


In [2]:
env = make('hungry_geese')
env.reset();

32.7 µs ± 480 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
